## 概述
- Java 代码有很多种不同的运行方式
  - 在开发工具中运行
  - 可以双击执行 jar 文件运行
  - 可以在命令行中运行
  - 可以在网页中运行
  - 这些执行方式都离不开 JRE，也就是 Java 运行时环境

```
# C 语言写的 Hello world 程序的编译结果
# 最左列是偏移；中间列是给机器读的机器码；最右列是给人读的汇编代码
0x00:  55                    push   rbp
0x01:  48 89 e5              mov    rbp,rsp
0x04:  48 83 ec 10           sub    rsp,0x10
0x08:  48 8d 3d 3b 00 00 00  lea    rdi,[rip+0x3b] 
                                    ; 加载 "Hello, World!\n"
0x0f:  c7 45 fc 00 00 00 00  mov    DWORD PTR [rbp-0x4],0x0
0x16:  b0 00                 mov    al,0x0
0x18:  e8 0d 00 00 00        call   0x12
                                    ; 调用 printf 方法
0x1d:  31 c9                 xor    ecx,ecx
0x1f:  89 45 f8              mov    DWORD PTR [rbp-0x8],eax
0x22:  89 c8                 mov    eax,ecx
0x24:  48 83 c4 10           add    rsp,0x10
0x28:  5d                    pop    rbp
0x29:  c3                    ret
```

---

## 为什么 Java 要在虚拟机里运行？
- 语法非常复杂，抽象程度也很高，直接在硬件上运行程序并不现实

### 如何转换
- 设计一个面向 Java 语言特性的虚拟机，并通过编译器将 Java 程序转换成该虚拟机所能识别的指令序列，也称 Java 字节码。

```
# Java 写的 Helloworld 程序编译的编译结果
# 最左列是偏移；中间列是给虚拟机读的机器码；最右列是给人读的代码
0x00:  b2 00 02         getstatic java.lang.System.out
0x03:  12 03            ldc "Hello, World!"
0x05:  b6 00 04         invokevirtual java.io.PrintStream.println
0x08:  b1               return
```

- Java 虚拟机可以由硬件实现，但更为常见的是在各个现有平台（如 Windows_x64、Linux_aarch64）上提供软件实现。
- 意义在于，一旦一个程序被转换成 Java 字节码，那么它便可以在不同平台上的虚拟机实现里运行。就是我们常说的“一次编写，到处运行”
- 另一个好处是带来了一个托管环境（Managed Runtime），这个托管环境能够代替我们处理一些代码中冗长而且容易出错的部分。
- 此外，托管环境还提供了诸如数组越界、动态类型、安全权限等等的动态检测，使我们免于书写这些无关业务逻辑的代码。

---

## Java 虚拟机具体是怎样运行 Java 字节码的？
- 以标准 JDK 中的 HotSpot 虚拟机为例，从虚拟机以及底层硬件两个角度来解释
- 虚拟机角度看
  - Java 代码首先转换为 class 文件加载到虚拟机中
  - 加载后的 Java 类会被存放于方法区（Method Area）中
  - 实际运行时，虚拟机会执行方法区内的代码
  - 运行中，每当调用一个 java 方法，java 虚拟机会在当前线程的 java 方法栈中生成一个栈帧，用来存放局部变量以及字节码的操作数（虚拟机不要求栈帧内存空间连续）
  - 退出当前执行的方法时，虚拟机会弹出当前的栈帧，并将之舍弃

![image](lesson_01_01.png)

- 硬件角度看
  - 字节码无法直接执行，虚拟机需要将字节码翻译成机器码
  - HotSpot 中，翻译过程分为两种
    - 解释执行：逐条将字节码翻译成机器码并执行
      - 优势：无需等待编译
    - 即时编译（Just-In-Time compilation，JIT)：将一个方法中包含的字节码编译成机器码再执行
      - 实际运行速度更快
  - HotSpot 默认采用混合模式
    - 它会先解释执行字节码，而后将其中反复执行的热点代码，以方法为单位进行即时编译

![image](lesson_01_02.png)

---

## Java 虚拟机的运行效率究竟是怎么样的？


![image](lesson_01_02.png)